# Small script to compare years of inventories and extrapolate the emissions


It reads from netcdf files

In [ ]:
from math import floor
from pathlib import Path
import copy

import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
path = Path(r"C:\Users\coli\Documents\ZH-CH-emission\output_files\mapluft_rasters") 
file_name = "zurich_inside_swiss_Fsplit_100x100_mapLuft_{year}_v2024_v1.8.nc"
inventories = {year: path / file_name.format(year=year) for year in (2015, 2020, 2022)}

compounds = [
    'CO2_bio',
    'CO2_fos',
    'CO',
    'BC',
    'SO2',
    'NOx',
    'N2O',
    'CH4',
    ]
inventories

In [ ]:
dss = {year: xr.open_dataset(inventory) for year, inventory in inventories.items()}

In [ ]:
dss[2015].source_category.values

In [ ]:
# print the missing cats for each inventories in each year
# This makes sure we don't have discrepenacies between the inventories
all_cats = np.unique(np.concatenate([inv.source_category.values for inv in dss.values()]))
for year, inv in dss.items():
    print(year)
    print([cat for cat in all_cats if cat not in inv.source_category ])

In [ ]:
ds = dss[2015]
ds["emi_CO2_fos_total"]

In [ ]:
ds.source_category.values

In [ ]:
# Get the total emissions 
dfs_total = {}
for year, ds in dss.items():
    total_emissions = {}
    categories = ds.source_category.values
    for compound in compounds:
        total_emissions[compound] = ds[f"emi_{compound}_total"].sel(source_category=categories)
    df_tot = pd.DataFrame(total_emissions, index=categories)

    df_tot *= 1e-3 # kg to ton
    df_tot[['CO2_bio', "CO2_fos"]] *= 1e-3 # ton to kton
    df_tot[compounds].round(1)

    df_tot.loc["__total__"] = df_tot.loc[ds.source_category.values].sum()
    df_tot.loc["GNFR_F"] = df_tot.loc[['GNFR_F-cars', 'GNFR_F-two_wheels', 'GNFR_F-heavy_duty',
        'GNFR_F-light_duty']].sum()

    dfs_total[year] = df_tot

In [ ]:
plt.style.use('default')

substances = compounds
fig, axes = plt.subplots(nrows=len(substances), figsize=(8,12), gridspec_kw={'hspace': 0.0}, sharex=True)
categories = [
        #"c1301_Personenwagen_Emissionen_Kanton",
        #"c1302_Lastwagen_Emissionen_Kanton",
        #"c1303_Motorraeder_Emissionen_Kanton",
        #"c1304_Linienbusse_Emissionen_Kanton",
        #"c1305_Trolleybusse_Emissionen_Kanton",
        #"c1306_StartStopTankatmung_Emissionen_Kanton",
        #"c1307_Lieferwagen_Emissionen_Kanton",
        #"c1308_Reisebusse_Emissionen_Kanton",
]
categories = [
    "GNFR_A",
    "GNFR_B",
    "GNFR_C",
    "GNFR_F",
    #"GNFR_G",
    "GNFR_I",
    "GNFR_J",
    "GNFR_K",
    #"GNFR_L",
    "GNFR_O",
    "GNFR_R",
    "__total__",
]
offsets = {
    2015: -0.2,
    2020: 0,
    2022: 0.2,

}
# Use some fancy colors
colors = {
    2015: '#1b9e77',
    2020: '#d95f02',
    2022: '#7570b3',
}

for i, substance in enumerate(substances):
    ax = axes[i]
    for year in [2015, 2020, 2022]:
        df_tot = dfs_total[year]

        if substance not in df_tot:
            continue
        # Add missing categories to the df as 0
        for cat in categories:
            if cat not in df_tot.index:
                df_tot.loc[cat] = 0
        # Plot a bar for each year
        x_axis = np.arange(len(categories)) + offsets[year]
        ax.bar(
            x_axis,
            df_tot[substance][categories],
            width=0.2,
            label=year,
            color=colors[year],
        )

    # No ticks 
    ax.set_xticks([])

    # Units 
    unit = 'kt' if substance == 'CO2' else 't'
    ax.set_ylabel(f"{substance} [{unit}]")


    
# just align the last column of axes:
fig.align_ylabels(axes)

ax.set_xticks(np.arange(len(categories)))
ax.set_xticklabels(categories)
# Rotate 
ax.set_xticklabels(categories, rotation=45)
ax.legend()
axes[0].set_title("Emissions by year, category, compound for zurich and surroundings (mapluft + swiss inventory)")

fig.savefig(path / "emissions_by_year_category_compound.png", dpi=300, bbox_inches='tight')
fig.tight_layout()
plt.show()